In [5]:
plaintext = 'Meet_NataSHA_which_is_not_a_SHA_although'
full_cipher = "ae055b48d8fa60bc337ff846ee88fe33c7e026a5ea54dbb59814c68265540cef1c183ef7465536868c4febe7e2f0a6d43110d37576535b8518eaa4b7ce3ac3722816062755aa8b5ed82eadf76e8af6f5"
ciphertext_hex = full_cipher[:80]
second_block_cipher = full_cipher[80:]

In [1]:
from natasha import ENC, RF, DEC
from tqdm import tqdm

In [7]:
byte_input = bytes(plaintext, "utf-8")
byte_ciph = bytes.fromhex(ciphertext_hex)

In [12]:
def sha_hash_24(text):
    val_map = dict()
    L_orig, R_orig = text[:20], text[20:]
    for k in range(2**24):
        key = k.to_bytes(3, byteorder="big")
        k0, k1,k2 = key[0:1], key[1:2], key[2:3]
        R, L = RF(L_orig, R_orig, k0)
        R, L = RF(L, R, k1)
        R, _ = RF(L, R, k2)
        val_map[R] = key
    return val_map

def findkeys(plaintext, other_map):
    L_orig, R_orig = plaintext[:20], plaintext[20:]
    for k in tqdm(range(2**24)):
        k0 = k & ((1 << 8) - 1)
        k1 = (k & ((1 << 16) - 1)) >> 8
        k2 = (k & ((1 << 24) - 1)) >> 16
        k0, k1, k2 = k0.to_bytes(1, byteorder="big"), k1.to_bytes(1, byteorder="big"), k2.to_bytes(1, byteorder="big")
        R, L = RF(L_orig, R_orig, k0)
        R, L = RF(L, R, k1)
        R, _ = RF(L, R, k2)
        if R in other_map:
            return k0 + k1 + k2, other_map[R]


def sha_hash_16(cipher_text_in_bytes):
    val_maps = dict()
    R_orig, L_orig = cipher_text_in_bytes[:20], cipher_text_in_bytes[20:]
    for k in tqdm(range(2**16)):
        key = k.to_bytes(2, byteorder="big")
        k4, k5 = key[0:1], key[1:2]
        R, L = RF(R_orig, L_orig, k5)
        R, _ = RF(L, R, k4)
        val_maps[R] = key
    return val_maps


def brute_force(first, second):
    for keys in second:
        if keys in first:
            k02 = first[keys]
            k45 = second[keys]
            print("success")

    for k in range(2**8):
        k3 = k.to_bytes(1, byteorder="big")
        key = k02 + k3 + k45
        if ENC(byte_input, key) == byte_ciph:
            print(key)
            return key
    


In [ ]:
l1_map = sha_hash_24(byte_input)

In [13]:
l2_map = sha_hash_16(byte_ciph)

k012, k45 = findkeys(byte_input, l2_map)

 81%|████████  | 13562171/16777216 [09:27<02:14, 23910.06it/s]


In [64]:
sec_cipher = bytes.fromhex(second_block_cipher)
DEC(sec_cipher, byte_key)

b'_we_use_it_not_to_hash_but_to_encrypt...'

In [15]:
byte_input.decode('utf-8')

'Meet_NataSHA_which_is_not_a_SHA_although'